In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import glob
import joblib

import librosa
import IPython.display as ipd


In [2]:
import glob

root = './free-spoken-digit-dataset-master/'
files = sorted(glob.glob(root + '**/*.wav', recursive=True))
print(len(files))
print(files[0])

3000
./free-spoken-digit-dataset-master/recordings/0_george_0.wav


In [3]:
%%time
import joblib as jl
dataset = []
def make_dataset(file, trim=True):
    dataset = []
    data,sr = librosa.load(file,sr=48000)

    if trim :
        data, area = librosa.effects.trim(data, top_db=10)
        data = librosa.util.fix_length(data, size=24000)

    mel = librosa.feature.melspectrogram(y=data, sr=sr)
    mfcc = librosa.feature.mfcc(y=data, sr=sr, n_mfcc=20)
    label = file.split('/')[-1].split('_')[0]
    dataset.append([mel, mfcc, label])
    return dataset

dataset = jl.Parallel(n_jobs=-1)(jl.delayed(make_dataset)(x) for x in files)


CPU times: user 1.59 s, sys: 329 ms, total: 1.92 s
Wall time: 1min


In [4]:
df_ds = pd.DataFrame(np.array(dataset, dtype=object).squeeze(), columns=['mel','mfcc','label'])


In [5]:
x_mel = np.array(df_ds.mel.to_list())
x_mfcc = np.array(df_ds.mfcc.to_list())
y = np.array(df_ds.label.astype(int))
print(x_mel.shape, x_mfcc.shape, y.shape)

(3000, 128, 47) (3000, 20, 47) (3000,)


In [6]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x_mfcc,y,test_size=0.2,stratify=y,
                                                 shuffle=True, random_state=42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(2400, 20, 47) (600, 20, 47) (2400,) (600,)


In [7]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

2026-02-12 14:41:25.296286: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-12 14:41:25.822295: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-12 14:41:28.540893: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [8]:
# model_tf = tf.keras.Sequential([
#     tf.keras.layers.Input(shape=(20, 47)),
#     tf.keras.layers.Conv1D(32, kernel_size=3, padding='same'),
#     tf.keras.layers.ReLU(),
#     tf.keras.layers.MaxPooling1D(pool_size=2),
    
#     tf.keras.layers.Conv1D(64, kernel_size=3, padding='same'),
#     tf.keras.layers.ReLU(),
#     tf.keras.layers.MaxPooling1D(pool_size=3),
    
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(100),
#     tf.keras.layers.ReLU(),
#     tf.keras.layers.Dense(10, activation='softmax')
# ])


In [9]:
model_tf = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1, 47, 20)),
    tf.keras.layers.Conv2D(32, kernel_size=(1, 3), padding='same', use_bias=True),
    tf.keras.layers.ReLU(max_value=6),
    tf.keras.layers.Conv2D(64, kernel_size=(1, 3), padding='same', use_bias=True),
    tf.keras.layers.ReLU(max_value=6),
    
    tf.keras.layers.GlobalAveragePooling2D(),
    
    tf.keras.layers.Dense(64, use_bias=True),
    tf.keras.layers.ReLU(max_value=6),
    tf.keras.layers.Dense(10, activation=None),
    tf.keras.layers.Softmax()
])

2026-02-12 14:41:29.246471: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [10]:
# Build the model first
# model_tf.build((None, 1, 47, 20))

# Compile the model
model_tf.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [11]:
model_tf.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 1, 47, 32)      │         1,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 1, 47, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 1, 47, 64)      │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 1, 47, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 64)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_2 (ReLU)                  │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ softmax (Softmax)               │ (None, 10)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,970 (50.66 KB)

 Trainable params: 12,970 (50.66 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
x_trn, x_val, y_trn, y_val = train_test_split(x_train, y_train, 
                                                test_size=0.2, random_state=42)

In [13]:
test_dataset = tf.data.Dataset.from_tensor_slices((x_test,y_test))

In [14]:
# Reshape data to add channel dimension and reorder: (samples, 20, 47) -> (samples, 1, 47, 20)
x_trn_reshaped = x_trn[:, np.newaxis, :, :].transpose(0, 1, 3, 2)
x_val_reshaped = x_val[:, np.newaxis, :, :].transpose(0, 1, 3, 2)

# Create TensorFlow datasets with reshaped data
train_dataset = tf.data.Dataset.from_tensor_slices((x_trn_reshaped, y_trn))
val_dataset = tf.data.Dataset.from_tensor_slices((x_val_reshaped, y_val))

# Configure the datasets (shuffle, batch, prefetch for performance)
train_dataset = train_dataset.shuffle(buffer_size=len(x_trn)).batch(32).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

In [15]:
model_tf.fit(train_dataset, validation_data=val_dataset, epochs=50)

Epoch 1/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.1823 - loss: 2.3199 - val_accuracy: 0.3750 - val_loss: 1.9553
Epoch 2/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.4255 - loss: 1.6544 - val_accuracy: 0.5042 - val_loss: 1.3641
Epoch 3/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.5771 - loss: 1.2086 - val_accuracy: 0.5729 - val_loss: 1.1018
Epoch 4/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6609 - loss: 0.9803 - val_accuracy: 0.6979 - val_loss: 0.8847
Epoch 5/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7260 - loss: 0.8213 - val_accuracy: 0.7437 - val_loss: 0.7739
Epoch 6/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7651 - loss: 0.7000 - val_accuracy: 0.7333 - val_loss: 0.7040
Epoch 7/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7937 - loss: 0.6290 - val_accuracy: 0.8021 - val_loss: 0.5782
Epoch 8/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8313 - loss: 0.5306 - val_accuracy: 0.7542 - val_l

In [16]:
# Cell: Evaluate on Test Set
# Reshape test data the same way: (samples, 20, 47) -> (samples, 1, 47, 20)
x_test_reshaped = x_test[:, np.newaxis, :, :].transpose(0, 1, 3, 2)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test_reshaped, y_test)).batch(32)

# Get predictions
y_pred = model_tf.predict(test_dataset)
y_pred_classes = np.argmax(y_pred, axis=1)

# Calculate accuracy
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

test_accuracy = accuracy_score(y_test, y_pred_classes)
print(f'Test Accuracy: {test_accuracy:.4f}')

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Test Accuracy: 0.9450


In [17]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(model_tf)
tflite_model = converter.convert()

# Save the model
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpn0c6dfom/assets


INFO:tensorflow:Assets written to: /tmp/tmpn0c6dfom/assets


Saved artifact at '/tmp/tmpn0c6dfom'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1, 47, 20), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  135374867911632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135374867912784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135374867910672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135374867910288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135374867912016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135374867911440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135374867913168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135374867912592: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1770887518.467324   84773 tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
W0000 00:00:1770887518.467387   84773 tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2026-02-12 14:41:58.468803: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpn0c6dfom
2026-02-12 14:41:58.470439: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2026-02-12 14:41:58.470499: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpn0c6dfom
I0000 00:00:1770887518.476198   84773 mlir_graph_optimization_pass.cc:437] MLIR V1 optimization pass is not enabled
2026-02-12 14:41:58.479669: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2026-02-12 14:41:58.529332: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpn0c6dfom
2026-02-12 14:41:58.545815: I tensorflow/cc/saved_model/loader.cc:471] SavedModel 

In [28]:
model_tf.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 1, 47, 32)      │         1,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 1, 47, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 1, 47, 32)      │         3,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 1, 47, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 1, 23, 32)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_2 (ReLU)                  │ (None, 1, 23, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 32)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │         4,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_3 (ReLU)                  │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_4 (ReLU)                  │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_5 (ReLU)                  │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_6 (ReLU)                  │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │           170 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ softmax (Softmax)               │ (None, 10)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 60,944 (238.07 KB)

 Trainable params: 20,314 (79.35 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 40,630 (158.71 KB)

In [ ]:
from sklearn.metrics import classification_report
# Cell: Detailed Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes, target_names=[str(i) for i in range(10)]))

print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, y_pred_classes)
print(cm)

In [ ]:
# Cell: Visualize Confusion Matrix
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=range(10), yticklabels=range(10))
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix - Audio Classification')
plt.tight_layout()
plt.show()

In [ ]:
model_tf.export("./tf_model")

In [ ]:
input_samples = []

In [ ]:
# Cell: Predict on New Audio File
def predict_digit(audio_file_path):
    """
    Predict digit from a new audio file
    
    Args:
        audio_file_path: path to .wav or .mp3 file
    
    Returns:
        predicted_digit: the predicted digit (0-9)
        confidence: prediction confidence
    """
    # Load and process audio
    data, sr = librosa.load(audio_file_path)
    print("Sample Rate: ", sr)
    print("data length", data.shape)
    print(data)
    # Trim silence and fix length
    data, _ = librosa.effects.trim(data, top_db=10)
    data = librosa.util.fix_length(data, size=24000)
    print("data length", data.shape)
    print(data)
    
    # Extract MFCC features
    mfcc = librosa.feature.mfcc(y=data, sr=sr, n_mfcc=20)
    
    # Reshape for model input (add batch dimension)
    mfcc_input = np.expand_dims(mfcc, axis=0)
    print("mfcc_input shape:", mfcc_input.shape)
    input_samples.append(mfcc_input)
    # Make prediction
    prediction = model_tf.predict(mfcc_input)
    predicted_digit = np.argmax(prediction[0])
    confidence = np.max(prediction[0])
    
    print(f"Audio File: {audio_file_path}")
    print(f"Predicted Digit: {predicted_digit}")
    print(f"Confidence: {confidence:.4f}")
    print(f"All probabilities: {prediction[0]}")
    
    return predicted_digit, confidence

# Example usage:
# predicted_digit, confidence = predict_digit('./test_audio.wav')

In [ ]:
# Cell: Predict on New Audio File
def predict_digit(audio_file_path):
    """
    Predict digit from a new audio file
    
    Args:
        audio_file_path: path to .wav or .mp3 file
    
    Returns:
        predicted_digit: the predicted digit (0-9)
        confidence: prediction confidence
    """
    # Load and process audio with same parameters as training
    data, sr = librosa.load(audio_file_path, sr=48000)
    print("Sample Rate: ", sr)
    print("Original data length:", data.shape)
    
    # Trim silence and fix length (same as training)
    data, _ = librosa.effects.trim(data, top_db=10)
    data = librosa.util.fix_length(data, size=24000)
    print("Processed data length:", data.shape)
    
    # Extract MFCC features (same parameters as training)
    mfcc = librosa.feature.mfcc(y=data, sr=sr, n_mfcc=20)
    print("MFCC shape:", mfcc.shape)  # Should be (20, 47)
    
    # Reshape for model input: (20, 47) -> (1, 1, 47, 20)
    # Step 1: Transpose to (47, 20)
    # Step 2: Add batch and channel dimensions to get (1, 1, 47, 20)
    mfcc_input = mfcc.T[np.newaxis, np.newaxis, :, :]
    input_samples.append(mfcc_input)
    print("Model input shape:", mfcc_input.shape)  # Should be (1, 1, 47, 20)
    
    # Make prediction
    prediction = model_tf.predict(mfcc_input, verbose=0)
    predicted_digit = np.argmax(prediction[0])
    confidence = np.max(prediction[0])
    
    print(f"\nAudio File: {audio_file_path}")
    print(f"Predicted Digit: {predicted_digit}")
    print(f"Confidence: {confidence:.4f}")
    print(f"All probabilities: {prediction[0]}")
    
    return predicted_digit, confidence

# Example usage:
# predicted_digit, confidence = predict_digit('./free-spoken-digit-dataset-master/recordings/7_nicolas_24.wav')

In [ ]:
data, sr = librosa.load('./free-spoken-digit-dataset-master/recordings/7_nicolas_24.wav')

In [ ]:
predicted_digit, confidence = predict_digit('./free-spoken-digit-dataset-master/recordings/7_nicolas_24.wav')

In [ ]:
input_samples

In [ ]:
print(input_samples[0].shape)
input_samples[0].flatten()

E:\audio_classification\ml-audio-classification\free-spoken-digit-dataset-master\recordings\7_nicolas_24.wav

In [ ]:
import tensorflow as tf
loaded = tf.saved_model.load('tf_model')
print("Signatures:", loaded.signatures.keys())  # Should show 'serving_default'

infer = loaded.signatures['serving_default']
print("Input:", infer.structured_input_signature)
print("Output:", infer.structured_outputs)




In [ ]:
tf_model = tf.saved_model.load('E:/audioSDK/build/tf_model')

In [ ]:
import numpy as np
import librosa

# Load audio file (mono, float32 in range [-1, 1])
# audio, sample_rate = librosa.load('./free-spoken-digit-dataset-master/recordings/7_nicolas_24.wav', sr=48000)
audio, sample_rate = librosa.load('./free-spoken-digit-dataset-master/recordings/7_nicolas_24.wav', sr=48000)
# trim audio with 10db
audio_trimmed, _ = librosa.effects.trim(audio, top_db=10)
data = librosa.util.fix_length(audio, size=24000)
# play using ipython display
import IPython.display as ipd
ipd.display(ipd.Audio(data, rate=sample_rate))

# Now indata is a numpy array of int32, matching your C++ buffer

In [ ]:
mfcc = librosa.feature.mfcc(y=data, sr=sample_rate, n_mfcc=20)

In [ ]:
# predict with mfcc
mfcc_input = np.expand_dims(mfcc, axis=0)
prediction = model_tf.predict(mfcc_input)
predicted_digit = np.argmax(prediction[0])
predicted_digit

In [ ]:
prediction

In [ ]:
mfcc

In [ ]:
# test with mfcc values convert to shape of mfcc input
mfcc_test = [-343.502, -343.502, -343.502, -343.502, -356, -356, -356, -356, -367.648, -367.648, -367.648, -363.78, -363.78, -363.78, -363.78, -368.221, -368.221, -368.221, -368.221, -372.439, -372.439, -372.439, -383.668, -383.668, -383.668, -383.668, -397.964, -397.964, -397.964, -409.54, -409.54, -409.54, -409.54, -423.221, -423.221, -423.221, -423.221, -435.175, -435.175, -435.175, -444.922, -444.922, -444.922, -444.922, -449.241, -449.241, -449.241,
314.023, 314.023, 314.023, 314.023, 330.864, 330.864, 330.864, 330.864, 332.062, 332.062, 332.062, 333.002, 333.002, 333.002, 333.002, 330.024, 330.024, 330.024, 330.024, 326.925, 326.925, 326.925, 317.436, 317.436, 317.436, 317.436, 304.761, 304.761, 304.761, 292.1, 292.1, 292.1, 292.1, 278.704, 278.704, 278.704, 278.704, 268.108, 268.108, 268.108, 257.247, 257.247, 257.247, 257.247, 249.06, 249.06, 249.06,
-3.05876, -3.05876, -3.05876, -3.05876, -5.48286, -5.48286, -5.48286, -5.48286, -6.05955, -6.05955, -6.05955, -11.1164, -11.1164, -11.1164, -11.1164, -9.56061, -9.56061, -9.56061, -9.56061, -8.30577, -8.30577, -8.30577, -5.85852, -5.85852, -5.85852, -5.85852, -3.72511, -3.72511, -3.72511, -8.00845, -8.00845, -8.00845, -8.00845, -9.33021, -9.33021, -9.33021, -9.33021, -7.56807, -7.56807, -7.56807, -8.98477, -8.98477, -8.98477, -8.98477, -8.55922, -8.55922, -8.55922,
-24.8763, -24.8763, -24.8763, -24.8763, -49.1603, -49.1603, -49.1603, -49.1603, -67.4007, -67.4007, -67.4007, -71.1666, -71.1666, -71.1666, -71.1666, -69.3151, -69.3151, -69.3151, -69.3151, -67.0097, -67.0097, -67.0097, -59.1323, -59.1323, -59.1323, -59.1323, -48.5832, -48.5832, -48.5832, -44.3884, -44.3884, -44.3884, -44.3884, -37.7093, -37.7093, -37.7093, -37.7093, -28.859, -28.859, -28.859, -21.8581, -21.8581, -21.8581, -21.8581, -9.88702, -9.88702, -9.88702,
2.9731, 2.9731, 2.9731, 2.9731, 14.4746, 14.4746, 14.4746, 14.4746, 17.6604, 17.6604, 17.6604, 18.1948, 18.1948, 18.1948, 18.1948, 19.5297, 19.5297, 19.5297, 19.5297, 21.898, 21.898, 21.898, 27.3241, 27.3241, 27.3241, 27.3241, 36.5817, 36.5817, 36.5817, 45.8127, 45.8127, 45.8127, 45.8127, 52.4078, 52.4078, 52.4078, 52.4078, 59.7071, 59.7071, 59.7071, 64.2103, 64.2103, 64.2103, 64.2103, 63.8667, 63.8667, 63.8667,
-14.8691, -14.8691, -14.8691, -14.8691, -14.5072, -14.5072, -14.5072, -14.5072, -1.89142, -1.89142, -1.89142, -0.339651, -0.339651, -0.339651, -0.339651, 1.81793, 1.81793, 1.81793, 1.81793, 5.3982, 5.3982, 5.3982, 8.17475, 8.17475, 8.17475, 8.17475, 13.4175, 13.4175, 13.4175, 20.9183, 20.9183, 20.9183, 20.9183, 22.9433, 22.9433, 22.9433, 22.9433, 24.7467, 24.7467, 24.7467, 25.2904, 25.2904, 25.2904, 25.2904, 16.3982, 16.3982, 16.3982,
-36.9122, -36.9122, -36.9122, -36.9122, -39.791, -39.791, -39.791, -39.791, -39.9536, -39.9536, -39.9536, -38.7176, -38.7176, -38.7176, -38.7176, -37.9416, -37.9416, -37.9416, -37.9416, -35.6189, -35.6189, -35.6189, -33.9588, -33.9588, -33.9588, -33.9588, -32.1821, -32.1821, -32.1821, -31.742, -31.742, -31.742, -31.742, -32.3286, -32.3286, -32.3286, -32.3286, -34.948, -34.948, -34.948, -29.3435, -29.3435, -29.3435, -29.3435, -22.8584, -22.8584, -22.8584,
10.0126, 10.0126, 10.0126, 10.0126, 11.0269, 11.0269, 11.0269, 11.0269, 12.2151, 12.2151, 12.2151, 13.9144, 13.9144, 13.9144, 13.9144, 11.8508, 11.8508, 11.8508, 11.8508, 9.09049, 9.09049, 9.09049, 6.35979, 6.35979, 6.35979, 6.35979, 2.70098, 2.70098, 2.70098, -1.20129, -1.20129, -1.20129, -1.20129, 0.847983, 0.847983, 0.847983, 0.847983, -2.21395, -2.21395, -2.21395, 5.69517, 5.69517, 5.69517, 5.69517, 14.2307, 14.2307, 14.2307,
34.9771, 34.9771, 34.9771, 34.9771, 34.629, 34.629, 34.629, 34.629, 33.5508, 33.5508, 33.5508, 34.4677, 34.4677, 34.4677, 34.4677, 32.6781, 32.6781, 32.6781, 32.6781, 26.4708, 26.4708, 26.4708, 18.7072, 18.7072, 18.7072, 18.7072, 11.0299, 11.0299, 11.0299, 9.64262, 9.64262, 9.64262, 9.64262, 15.6341, 15.6341, 15.6341, 15.6341, 16.6538, 16.6538, 16.6538, 17.0487, 17.0487, 17.0487, 17.0487, 10.7409, 10.7409, 10.7409,
-15.9883, -15.9883, -15.9883, -15.9883, -18.1385, -18.1385, -18.1385, -18.1385, -23.8931, -23.8931, -23.8931, -24.6127, -24.6127, -24.6127, -24.6127, -24.498, -24.498, -24.498, -24.498, -28.5652, -28.5652, -28.5652, -33.8341, -33.8341, -33.8341, -33.8341, -37.3701, -37.3701, -37.3701, -34.4911, -34.4911, -34.4911, -34.4911, -29.5546, -29.5546, -29.5546, -29.5546, -25.0943, -25.0943, -25.0943, -28.895, -28.895, -28.895, -28.895, -34.5879, -34.5879, -34.5879,
-25.3912, -25.3912, -25.3912, -25.3912, -31.5148, -31.5148, -31.5148, -31.5148, -33.834, -33.834, -33.834, -34.5997, -34.5997, -34.5997, -34.5997, -34.7492, -34.7492, -34.7492, -34.7492, -33.9352, -33.9352, -33.9352, -31.2751, -31.2751, -31.2751, -31.2751, -27.5553, -27.5553, -27.5553, -23.4665, -23.4665, -23.4665, -23.4665, -23.8957, -23.8957, -23.8957, -23.8957, -23.2106, -23.2106, -23.2106, -24.2785, -24.2785, -24.2785, -24.2785, -23.5249, -23.5249, -23.5249,
11.3821, 11.3821, 11.3821, 11.3821, 12.0312, 12.0312, 12.0312, 12.0312, 14.4456, 14.4456, 14.4456, 14.7293, 14.7293, 14.7293, 14.7293, 14.3459, 14.3459, 14.3459, 14.3459, 18.2445, 18.2445, 18.2445, 24.5232, 24.5232, 24.5232, 24.5232, 27.6233, 27.6233, 27.6233, 30.0357, 30.0357, 30.0357, 30.0357, 25.6792, 25.6792, 25.6792, 25.6792, 19.3395, 19.3395, 19.3395, 19.1657, 19.1657, 19.1657, 19.1657, 17.844, 17.844, 17.844,
5.79131, 5.79131, 5.79131, 5.79131, 5.94488, 5.94488, 5.94488, 5.94488, 13.6408, 13.6408, 13.6408, 14.9835, 14.9835, 14.9835, 14.9835, 16.5035, 16.5035, 16.5035, 16.5035, 19.7326, 19.7326, 19.7326, 23.5704, 23.5704, 23.5704, 23.5704, 21.204, 21.204, 21.204, 19.6522, 19.6522, 19.6522, 19.6522, 14.2491, 14.2491, 14.2491, 14.2491, 8.67062, 8.67062, 8.67062, 10.1736, 10.1736, 10.1736, 10.1736, 13.0379, 13.0379, 13.0379,
-20.3889, -20.3889, -20.3889, -20.3889, -20.7152, -20.7152, -20.7152, -20.7152, -17.3163, -17.3163, -17.3163, -15.1633, -15.1633, -15.1633, -15.1633, -13.436, -13.436, -13.436, -13.436, -13.7927, -13.7927, -13.7927, -13.4285, -13.4285, -13.4285, -13.4285, -15.0306, -15.0306, -15.0306, -20.1432, -20.1432, -20.1432, -20.1432, -25.3237, -25.3237, -25.3237, -25.3237, -22.8517, -22.8517, -22.8517, -16.6829, -16.6829, -16.6829, -16.6829, -8.81239, -8.81239, -8.81239,
0.377243, 0.377243, 0.377243, 0.377243, 1.9161, 1.9161, 1.9161, 1.9161, -1.03689, -1.03689, -1.03689, -0.505921, -0.505921, -0.505921, -0.505921, -2.10267, -2.10267, -2.10267, -2.10267, -5.07917, -5.07917, -5.07917, -7.17135, -7.17135, -7.17135, -7.17135, -3.85189, -3.85189, -3.85189, -6.89115, -6.89115, -6.89115, -6.89115, -9.06257, -9.06257, -9.06257, -9.06257, -3.36003, -3.36003, -3.36003, 0.500027, 0.500027, 0.500027, 0.500027, 0.617363, 0.617363, 0.617363,
17.1748, 17.1748, 17.1748, 17.1748, 17.2424, 17.2424, 17.2424, 17.2424, 19.282, 19.282, 19.282, 15.9493, 15.9493, 15.9493, 15.9493, 11.9354, 11.9354, 11.9354, 11.9354, 10.3424, 10.3424, 10.3424, 7.88, 7.88, 7.88, 7.88, 11.2123, 11.2123, 11.2123, 13.9164, 13.9164, 13.9164, 13.9164, 17.3808, 17.3808, 17.3808, 17.3808, 18.4524, 18.4524, 18.4524, 14.7623, 14.7623, 14.7623, 14.7623, 8.27111, 8.27111, 8.27111,
-1.49214, -1.49214, -1.49214, -1.49214, -6.05715, -6.05715, -6.05715, -6.05715, -5.47521, -5.47521, -5.47521, -9.14267, -9.14267, -9.14267, -9.14267, -11.8191, -11.8191, -11.8191, -11.8191, -11.3349, -11.3349, -11.3349, -10.8255, -10.8255, -10.8255, -10.8255, -9.70514, -9.70514, -9.70514, -5.53222, -5.53222, -5.53222, -5.53222, -0.0607634, -0.0607634, -0.0607634, -0.0607634, -2.24638, -2.24638, -2.24638, -6.26682, -6.26682, -6.26682, -6.26682, -11.8559, -11.8559, -11.8559,
-5.35388, -5.35388, -5.35388, -5.35388, -11.4012, -11.4012, -11.4012, -11.4012, -19.8155, -19.8155, -19.8155, -19.2656, -19.2656, -19.2656, -19.2656, -19.0908, -19.0908, -19.0908, -19.0908, -20.0239, -20.0239, -20.0239, -17.3338, -17.3338, -17.3338, -17.3338, -14.8216, -14.8216, -14.8216, -12.8681, -12.8681, -12.8681, -12.8681, -11.7762, -11.7762, -11.7762, -11.7762, -12.3246, -12.3246, -12.3246, -13.2712, -13.2712, -13.2712, -13.2712, -21.1152, -21.1152, -21.1152,
5.1184, 5.1184, 5.1184, 5.1184, 5.15278, 5.15278, 5.15278, 5.15278, 1.39852, 1.39852, 1.39852, 3.02819, 3.02819, 3.02819, 3.02819, 5.30512, 5.30512, 5.30512, 5.30512, 3.10652, 3.10652, 3.10652, 3.2259, 3.2259, 3.2259, 3.2259, 4.97816, 4.97816, 4.97816, 7.31651, 7.31651, 7.31651, 7.31651, 4.31503, 4.31503, 4.31503, 4.31503, 5.03482, 5.03482, 5.03482, 2.58673, 2.58673, 2.58673, 2.58673, -5.96565, -5.96565, -5.96565,
-4.55583, -4.55583, -4.55583, -4.55583, -0.326159, -0.326159, -0.326159, -0.326159, 5.30831, 5.30831, 5.30831, 3.08599, 3.08599, 3.08599, 3.08599, 6.1697, 6.1697, 6.1697, 6.1697, 7.1047, 7.1047, 7.1047, 4.65248, 4.65248, 4.65248, 4.65248, -0.359649, -0.359649, -0.359649, 2.52646, 2.52646, 2.52646, 2.52646, 1.00671, 1.00671, 1.00671, 1.00671, 1.69803, 1.69803, 1.69803, 0.458092, 0.458092, 0.458092, 0.458092, -0.466117, -0.466117, -0.466117,
]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Plot MFCCs and their difference
fig, axs = plt.subplots(1, 3, figsize=(21, 5))

# First MFCC
im0 = axs[0].imshow(mfcc.T, aspect='auto', origin='lower')
axs[0].set_title('MFCC (from file)')
plt.colorbar(im0, ax=axs[0])

# Second MFCC
mfcc_input_test = np.array(mfcc_test).reshape(1,20,47)
im1 = axs[1].imshow(mfcc_input_test[0].T, aspect='auto', origin='lower')
axs[1].set_title('MFCC (from ml mod)')
plt.colorbar(im1, ax=axs[1])

# Difference
diff = np.abs(mfcc.T - mfcc_input_test[0].T)
im2 = axs[2].imshow(diff, aspect='auto', origin='lower', cmap='hot')
axs[2].set_title('Absolute Difference')
plt.colorbar(im2, ax=axs[2])

plt.tight_layout()
plt.show()

In [ ]:
mfcc_input_test = np.array(mfcc_test).reshape(1,20,47)
prediction = model_tf.predict(mfcc_input_test)
predicted_digit = np.argmax(prediction[0])
predicted_digit

In [ ]:
mfcc.shape

In [ ]:
# predict with mfcc
mfcc_input = np.expand_dims(mfcc, axis=0)
prediction = model_tf.predict(mfcc_input)
predicted_digit = np.argmax(prediction[0])
predicted_digit

In [ ]:
mfcc.flatten()[:20]

In [ ]:
# for 256 values add new lines
with open('sample.txt', 'w') as f:
    for i in range(0, len(audio), 256):
        line = ', '.join(str(x) for x in audio[i:i+256])
        f.write(line + '\n\n\n')

In [ ]:
import tensorflow as tf

# Load TFLite model
model = tf.lite.Interpreter(model_path="ti_model.tflite")
model.allocate_tensors()

# Get tensor details
tensor_details = model.get_tensor_details()
input_details = model.get_input_details()
output_details = model.get_output_details()

# Create summary table
print("=" * 80)
print("Model: TFLite Model")
print("=" * 80)
print(f"{'Layer (type)':<40} {'Output Shape':<25} {'Param #':<15}")
print("=" * 80)

total_params = 0

for tensor in tensor_details:
    name = tensor['name']
    shape = tensor['shape']
    
    # Calculate parameters (for weight tensors)
    if len(shape) > 0:
        params = 1
        for dim in shape:
            params *= dim
        total_params += params
    else:
        params = 0
    
    print(f"{name:<40} {str(list(shape)):<25} {params:<15,}")

print("=" * 80)
print(f"Total params: {total_params:,}")
print("=" * 80)

# Print input/output info
print("\nInput layers:")
for inp in input_details:
    print(f"  {inp['name']}: {inp['shape']} ({inp['dtype'].__name__})")

print("\nOutput layers:")
for out in output_details:
    print(f"  {out['name']}: {out['shape']} ({out['dtype'].__name__})")

In [ ]:
import flatbuffers
import tensorflow as tf
from tflite import Model

# Load the .tflite file
with open("model.tflite", "rb") as f:
    buf = f.read()

# Get the model object (correct usage)
model = Model.GetRootAsModel(buf, 0)

print("Model version:", model.Version())
print("Number of subgraphs:", model.SubgraphsLength())

# Inspect first tensor in first subgraph
subgraph = model.Subgraphs(0)
tensor = subgraph.Tensors(0)
print("Tensor name:", tensor.Name().decode("utf-8"))
print("Tensor shape:", [tensor.Shape(i) for i in range(tensor.ShapeLength())])
